In [ ]:
import pandas as pd
import dill
import numpy as np
import numpy.linalg as la
pd.__version__

In [ ]:
dill.__version__

In [ ]:
#Q1a
raw_A1 = np.array([[62.2, 171.1, 17, 36.8],\
                    [52.2, 162.6, 19, 36.5],\
                    [72.3, 178.2, 22, 36.7],\
                    [80.8, 185.2, 24, 37.9],\
                    [72.5, 178.3, 22, 37.8]], dtype = float)


A1 = raw_A1[:, 0:3]

print(raw_A1)
print(A1)

In [ ]:
#Q1b

# The closer the 2 vectors are, the larger their cosine similarity
# Measure cosine similarity of each person (row) and compare

In [ ]:
#Q1c
def my_norm(x):
    sum = 0
    for i in range(len(x)):
        sum += x[i] ** 2
    return np.sqrt(sum)

def my_dot(x, y):
    if len(x) != len(y):
        print("Array have different dimensions, error.")
        return -1
    
    dot = 0
    for i in range(len(x)):
        dot += x[i] * y[i]
    return dot

def my_cosSimilarity(x, y):
    return my_dot(x, y) / (my_norm(x) * my_norm(y))

In [ ]:
#Q1d
Bob = A1[0,:]
Alice = A1[1,:]

print("BOB:\n", "My norm: ", my_norm(Bob), "\nNumpy norm: ", la.norm(Bob), sep="", end="\n\n")
print("ALICE:\n", "My norm: ", my_norm(Alice), "\nNumpy norm: ", la.norm(Alice), sep="", end="\n\n")
print("DOT:\n", "My dot: ", my_dot(Bob, Alice), "\nNumpy dot: ", np.dot(Bob, Alice), sep="", end="\n\n")
print("Cosine Similarity:", my_cosSimilarity(Bob, Alice))

In [ ]:
#Q1e
A1_norm = np.zeros(A1.shape)
(people, features) = A1.shape
for i in range(people):
    A1_norm[i] = A1[i]
    A1_norm[i] /= my_norm(A1[i])

print("A1_norm:\n", A1_norm, sep="")

In [ ]:
#Q1f
Bob = A1_norm[0,:]
Alice = A1_norm[1,:]

print("BOB:\n", "My norm: ", my_norm(Bob), "\nNumpy norm: ", la.norm(Bob), sep="", end="\n\n")
print("ALICE:\n", "My norm: ", my_norm(Alice), "\nNumpy norm: ", la.norm(Alice), sep="", end="\n\n")
print("DOT:\n", "My dot: ", my_dot(Bob, Alice), "\nNumpy dot: ", np.dot(Bob, Alice), sep="", end="\n\n")
print("Cosine Similarity:", my_cosSimilarity(Bob, Alice))

In [ ]:
#Q1g
Unknown = A1_norm[4,:]
similarity_vector = np.zeros(4)
for i in range(4):
    person = A1_norm[i,:]
    similarity_vector[i] = my_cosSimilarity(Unknown, person)
    
print("Similarity Vector:", similarity_vector)

In [ ]:
#Q1h

# Person 3 is the most similar to the unknown person because
# his similarity vector with the unknown person is the highest.

In [ ]:
#Q2a

A = np.array([[1, 1, 1],
             [1, 1, 1],
             [1, 1, -1],
             [1, -1, -1]], dtype = float)

B = np.array([[3, -1, -1/2],
             [1, 2, -2],
             [1, 1, 7/2],
             [0, 1, 0]], dtype = float)

def orth_set(matrix):
    QTQ = matrix.T @ matrix

    (rows, n) = matrix.shape
    for i in range(n):
        for j in range(n):
            if i != j:
                if QTQ[i][j] != 0:
                    return False
    
    return True
    
print("A:", orth_set(A))
print("B:", orth_set(B))

# B's off-diagonal elements of its transpose x itself are all 0, while A's arent't
# hence B is orthogonal while A isn't

In [ ]:
#Q2b

# Theorem 5:
# B is orthogonal so weights in a linear combination can be computed easily
# c_j = <y, u_j> / <u_j, u_j>

y1 = np.array([2.5, 11, -7.5, 2])

def weights_orth(A, b_row):
    (rows, cols) = A.shape
    x = np.zeros((cols, 1))

    for j in range(cols):
        u = A[:,j]
        c = np.dot(b_row, u) / np.dot(u, u)
        x[j] = c
    
    return x

x = weights_orth(B, y1)
print("x:\n", x)
    



In [ ]:
#Q2c
(rows, cols) = A.shape
y2 = np.zeros((1, rows)) # y2 is declared as row vector here or np will throw error below
weights = [1, 2, -3]

for i in range(cols):
    ai = A[:,i]
    y2 += weights[i] * ai

print("y2:", y2, end="\n\n")

# find x using Theorem 5
x = np.zeros((cols, 1))
for j in range(cols):
    u = A[:,j]
    c = np.dot(y2, u) / np.dot(u, u)
    x[j] = c

print("x using Theorem 5:\n", x, end="\n\n")

# Ax
Ax = A @ x
print("Ax:\n", Ax)

# correct value for x should be [1, 2, -3]

In [ ]:
#Q2d

# normalise B
BnT = np.zeros(B.shape).T
BT = B.T

(rows, cols) = B.shape    
for i in range(cols):
    BnT[i] = BT[i]
    BnT[i] /= my_norm(BT[i])
    
Bn = BnT.T
print("Bn:\n", Bn, sep="", end="\n\n")

# show Bn is an orthonormal set
qtq_raw = BnT @ Bn
qtq = np.around(qtq_raw, 2)
print("Result of Bn transposed x Bn is an identity matrix:\n", qtq,\
     "\nhence Bn is an orthonormal set", sep="", end="\n\n")

# solve y1 = Bn * xn
xn = weights_orth(Bn, y1)
print("xn:\n", xn, sep="")

# each of xn's weights are the corresponding x's weights * length of B's corresponding columns

In [ ]:
#Q2e

# projection of y1 on cols of B
(rows, cols) = B.shape
proj_B_y1 = np.zeros((cols, rows))

for i in range(cols):
    u = B[:,i]
    proj_B_y1[i] = ((np.dot(y1, u) / np.dot(u, u)) * u)
    print(f"proj_B_y1 for column {i+1}:\n", proj_B_y1[i], sep="", end="\n\n")

# error for each column = perpendicular dist. from y1 to projection
error = np.zeros(cols)
for i in range(cols):
    zi = y1 - proj_B_y1[i]
    error[i] = la.norm(zi)
    print(f"Error for column {i+1}:\n", error[i], sep="", end="\n\n")
    
# order of importance of Bn columns to approximate y1:
# lowest error to highest -> 3, 2, 1

In [ ]:
#Q2f
(cols, rows) = proj_B_y1.shape
least_error = (0, 0, 1000)

for i in range(cols):
    for j in range(cols):
        if i == j:
            continue
        appx = proj_B_y1[i] + proj_B_y1[j]
        z = y1 - appx
        error = (i+1, j+1, la.norm(z))
        
        if error[2] < least_error[2]:
            least_error = error

print(f"Columns {least_error[0]} and {least_error[1]} of B should be chosen.\nResidual error: {least_error[2]}")

In [ ]:
#Q3a

def qr_gram_schmidt(A):
    # The QR factorization using the classical Gram Schmidt process.
    # Input: an arbitrary mxp matrix.
    # Output: the Q and R factors of the input.

    # Obtain the column number.
    p = A.shape[1]

    # Initiate the Q and R factors.
    Q = A.copy() # Prevent A from being changed.
    R = np.zeros((p, p))

    # Iterate for each column of the input.
    for k in range(p):
        w = A[:, k]
        
        ##### Different part. #####
        for i in range(k):
            R[i, k] = Q[:, i].dot(w)

        for i in range(k):
            w = w - R[i, k] * Q[:, i]
        ###########################
        
        R[k, k] = la.norm(w)
        Q[:, k] = w / R[k, k]

    return Q, R

Q, R = qr_gram_schmidt(A)
print("Q:\n", Q, "\n\nR:\n", R, sep="")

In [ ]:
#Q3b 
# Characteristics of columns of Q:
# n-i rows in each column i are identical, starting from the top row
# where n is the total number of rows

print("Q'*Q:\n", np.around(Q.T @ Q, 6),\
      "\n\nQ*Q':\n", np.around(Q @ Q.T, 6), sep="", end="\n\n")

# Q*Q' is the projection matrix with column space of A
# aka. Q*Q' * [any vector] to project the vector onto A

In [ ]:
#Q3c
print("rank of A:", la.matrix_rank(A))

In [ ]:
#Q3d
aug_matrix = np.hstack((A,Q))
print("rank of [A Q]:", la.matrix_rank(aug_matrix))

# rank of [A Q] is the same as rank of A
# implies that Q has the same column space as A

In [ ]:
#Q3e
def proj_bySum(U, y):
    (m, n) = U.shape
    p = np.zeros(m)
    
    for i in range(n):
        u = U[:,i]
        p += np.dot(y, u) * u
    
    return p

def proj_byP(U, y):
    P = U @ U.T
    return P @ y
    

In [ ]:
#Q3f
y = np.array([1, 2, 3, 4])

p1 = proj_bySum(Q, y)
p2 = proj_byP(Q, y)

print("Method 1: proj = (y.u1)u1 + ... + (y.up)up\np =", p1, end="\n\n")
print("Method 2: proj = UU^Ty\np =", p2, end="\n\n")

z = y - p1
error = la.norm(y - p1)
print("Error:", error)